<a href="https://colab.research.google.com/github/najat-ttt/nasa-space-apps-challenge/blob/main/Exo_Hunter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Task**
Keras Model and LightGBM Model creation and Training with exoplanets dataset collected from **NASA** for **Exo-Hunter** project that will be submitted for **NSAC-2025** event.


#**01. Load the Dataset**

i. Connect Drive Storage with Colab

Drive link: https://drive.google.com/drive/folders/1flLIuTxZyQoBZqtD35ISzf8mZVcVKQBS?usp=drive_link

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


ii. Load the data from "/content/drive/MyDrive/Exo-Hunter/exoplanets_vs_false_FIXED.csv" into a pandas DataFrame.

In [2]:
from os import linesep
import pandas as pd

file_path = '/content/drive/MyDrive/Exo-Hunter/exoplanets_vs_false_FIXED.csv'
try:
    df = pd.read_csv(file_path)
    display(df.head())
    print("\n")
    display(df.info())
except FileNotFoundError:
    print(f"Error: File not found at {file_path}. Please check the file path and try again.")

,mission,object_name,disposition,period,planet_radius,star_temp,star_radius,star_mass,discovery_facility
0,ARCHIVE,PLANET_1734,CONFIRMED,3.943328,7.968167,5314.834165,0.969259,0.246349,UNKNOWN
1,Kepler,K00711.03,CONFIRMED,124.524522,2.690000,5497.000000,1.046000,0.988000,NaN
2,Kepler,K01501.01,CONFIRMED,2.617028,1.570000,4831.000000,0.724000,0.706000,NaN
3,ARCHIVE,PLANET_168,CONFIRMED,300.262512,13.237900,9170.291947,3.498023,2.619901,UNKNOWN
4,ARCHIVE,PLANET_1768,CONFIRMED,325.259434,13.519703,5557.471709,3.712298,1.821366,UNKNOWN




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16025 entries, 0 to 16024
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   mission             16025 non-null  object 
 1   object_name         16025 non-null  object 
 2   disposition         16025 non-null  object 
 3   period              15992 non-null  float64
 4   planet_radius       15647 non-null  float64
 5   star_temp           15735 non-null  float64
 6   star_radius         15646 non-null  float64
 7   star_mass           13242 non-null  float64
 8   discovery_facility  6013 non-null   object 
dtypes: float64(5), object(4)
memory usage: 1.1+ MB


None

# **02. Preprocessing the Dataset**
The dataset has been loaded and inspected. The next step is to preprocess this data by selecting relevant features, handling missing values, and scaling the numerical features, similar to the preprocessing done for the previous dataset. This includes identifying numerical and categorical features, imputing missing values, and applying scaling and one-hot encoding where appropriate, then splitting the data into training and validation sets.

In [3]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

# Define features and target columns
features = ['period', 'planet_radius', 'star_temp', 'star_radius', 'star_mass', 'discovery_facility']
target = 'disposition'

# Check if the target column exists in the DataFrame
if not all(col in df.columns for col in features+ [target]):
  missing = [col for col in features+ [target] if col not in df.columns]
  raise ValueError(f"Fixed CSV must contain the following columns: {missing}")

# Filter for relevant dispositions and map to numerical labels
df_filtered = df[df[target].isin(['CONFIRMED', 'FALSE POSITIVE'])]
df_filtered['label'] = df_filtered[target].apply(lambda x: 1 if x == 'CONFIRMED' else 0)

# Separate features and target
X = df_filtered[features]
y = df_filtered['label']

# Identify numerical and categorical features
numerical_features = ['period', 'planet_radius', 'star_temp', 'star_radius', 'star_mass']
categorical_features = ['discovery_facility']

# Create preprocessing pipelines for numerical and categorical features
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')), # Use median imputation for numerical features
    ('scaler', StandardScaler()) # Scale numerical features
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')), # Use constant imputation for categorical features
    ('onehot', OneHotEncoder(handle_unknown='ignore')) # One-hot encode categorical features
])

# Create a column transformer to apply different transformations to different columns
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='passthrough' # Keep the target column unchanged
)

# Apply preprocessing to the dataset
X_preprocessed = preprocessor.fit_transform(X)

print(f"X_preprocessed shape: {X_preprocessed.shape}, y_preprocessed shape: {y.shape}")
print(f"Processed data shape: {X_preprocessed.shape}")

# Split the dataset into training and testing sets
X_train, X_val, y_train, y_val = train_test_split(X_preprocessed, y, test_size=0.2, stratify=y, random_state=42)

print(f"Training data shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"Valiation data shape: {X_val.shape}, y_test shape: {y_val.shape}")

X_preprocessed shape: (16025, 7), y_preprocessed shape: (16025,)
Processed data shape: (16025, 7)
Training data shape: (12820, 7), y_train shape: (12820,)
Valiation data shape: (3205, 7), y_test shape: (3205,)


# **03. Model Creation and Training**
The fixed dataset has been preprocessed and split into training and validation sets. The next step is to define and train the model using the preprocessed training and validation data from this dataset. We will use two model for better comparison:

*   Basic Tabular **Keras** Model
*   **LightGBM**

---

# Keras Model


In [4]:
from tensorflow.keras import layers, models

def make_tabular_model(input_dim):
    model = models.Sequential([
        layers.Input(shape=(input_dim,)),
        layers.Dense(64, activation='relu'),
        layers.Dense(32, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Create the Keras model with new input dimension
input_dim = X_train.shape[1]
basic_keras_model = make_tabular_model(input_dim)

# Train the Keras model
print("Training Basic Keras Model with the dataset...")
history_keras_model = basic_keras_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=70, batch_size=32, verbose = 1)
print("Training Done!")

Training Basic Keras Model with the dataset...
Epoch 1/70
401/401 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7183 - loss: 0.5181 - val_accuracy: 0.7835 - val_loss: 0.3814
Epoch 2/70
401/401 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7957 - loss: 0.3658 - val_accuracy: 0.7991 - val_loss: 0.3678
Epoch 3/70
401/401 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8088 - loss: 0.3533 - val_accuracy: 0.8162 - val_loss: 0.3566
Epoch 4/70
401/401 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8226 - loss: 0.3408 - val_accuracy: 0.8324 - val_loss: 0.3457
Epoch 5/70
401/401 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8289 - loss: 0.3315 - val_accuracy: 0.8290 - val_loss: 0.3413
Epoch 6/70
401/401 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8364 - loss: 0.3232 - val_accuracy: 0.8296 - val_loss: 0.3416
Epoch 7/70
401/401 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8317 - loss: 0.3239 - val_accuracy: 0.8505 - val_loss: 0.3191
Epoch 8/70
401/401 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - acc

The basic Keras model has been trained on the dataset. The next step is to evaluate the trained Keras model on the validation set to get the accuracy and loss.

In [5]:
# Evaluate the basic Keras model on the fixed validation set
print("Evaluating Basic Keras Model with dataset...")
loss_basic_keras, accuracy_basic_keras = basic_keras_model.evaluate(X_val, y_val, verbose=0)

print(f"Basic Keras Model Evaluation - Loss: {loss_basic_keras:.4f}, Accuracy: {accuracy_basic_keras:.4f}")

Evaluating Basic Keras Model with dataset...
Basic Keras Model Evaluation - Loss: 0.2832, Accuracy: 0.8612


# LightGBM Model

In [6]:
!pip install LightGBM

In [7]:
import lightgbm as lgb

model_lgb = lgb.LGBMClassifier(
    random_state=42,
    n_estimators=1500,       # More trees since we're regularizing well
    learning_rate=0.025,     # Slightly lower
    num_leaves=20,           # Slight increase in complexity
    max_depth=5,             # One level deeper
    min_child_samples=40,    # Slightly less conservative
    subsample=0.85,          # Slight increase
    colsample_bytree=0.85,   # Slight increase
    reg_alpha=0.25,          # Slightly less regularization
    reg_lambda=0.25,         # Slightly less regularization
    force_col_wise=True
)

print("\nTraining LightGBM Model...")
model_lgb.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_val, y_val)],
    eval_metric=['logloss', 'accuracy'],
    callbacks=[
        lgb.early_stopping(150, verbose=False),  # More patience
        lgb.log_evaluation(100)
    ]
)
print("Training Done!")


Training LightGBM Model...
[LightGBM] [Info] Number of positive: 8020, number of negative: 4800
[LightGBM] [Info] Total Bins 1279
[LightGBM] [Info] Number of data points in the train set: 12820, number of used features: 7
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.625585 -> initscore=0.513323
[LightGBM] [Info] Start training from score 0.513323
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with p

The LightGBM model has been trained on the dataset. The next step is to evaluate the trained LightGBM model on the validation set to get the accuracy and loss (logloss).

In [8]:
from sklearn.metrics import accuracy_score, log_loss

# Evaluate the LightGBM model on the fixed validation set
print("Evaluating LightGBM Model with dataset...")
y_pred_lgb = model_lgb.predict(X_val)
y_pred_proba_lgb = model_lgb.predict_proba(X_val)[:, 1]
accuracy_lgb = accuracy_score(y_val, y_pred_lgb)
logloss_lgb = log_loss(y_val, y_pred_proba_lgb)

# Calculate logloss if there are both positive and negative classes in the validation set
# Check if y_val_fixed contains both 0 and 1
if len(y_val.unique()) > 1:
    logloss_lgb = log_loss(y_val, y_pred_proba_lgb)
    print(f"LightGBM Model Evaluation - Loss: {logloss_lgb:.4f}, Accuracy: {accuracy_lgb:.4f}")
else:
    print(f"LightGBM Model Evaluation - Loss: {logloss_lgb:.4f}, Accuracy: {accuracy_lgb:.4f}")

Evaluating LightGBM Model with dataset...


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


LightGBM Model Evaluation - Loss: 0.2088, Accuracy: 0.9164


# **04. Comparison between the Models**
Comparison of the validation loss and accuracy scores of the two models and print a conclusion based on the results.

In [9]:
# Compare the validation loss and accuracy scores

print("Comparison between the Models")
print(f"Basic Keras Model - Loss: {history_keras_model.history['val_loss'][-1]:.4f}, Accuracy: {history_keras_model.history['val_accuracy'][-1]:.4f}")
print(f"LightGBM Model - Loss: {logloss_lgb:.4f}, Accuracy: {accuracy_lgb:.4f}")

Comparison between the Models
Basic Keras Model - Loss: 0.2832, Accuracy: 0.8612
LightGBM Model - Loss: 0.2088, Accuracy: 0.9164


In [25]:
# Define the output file path
output_file_path = '/content/candidate_predictions_lgb.csv'

# Save the predictions DataFrame to a CSV file
predictions_df.to_csv(output_file_path, index=False)

print(f"LightGBM candidate predictions saved to: {output_file_path}")

LightGBM candidate predictions saved to: /content/candidate_predictions_lgb.csv


## Summary:

### Data Analysis Key Findings

* The new dataset contains 2663 candidate entries and 7 features after preprocessing.
* The LightGBM model successfully generated predictions for all 2663 candidates.
* The LightGBM model outputs both a probability score and a predicted class (0 or 1) per candidate.

### Insights or Next Steps

* The predictions from the LightGBM model can be used to identify potential candidates based on the predicted class or a chosen probability threshold.
* Further analysis could involve investigating the characteristics of candidates with high prediction scores for the 'CONFIRMED' class.

# Task
Predict on the dataset "/content/all_candidates_standard.csv" using the trained Keras and LightGBM models.

## Load the new dataset

### Subtask:
Load the data from "/content/all_candidates_standard.csv" into a dataframe.


**Reasoning**:
Load the data from the specified CSV file into a pandas DataFrame and display its head and info.



In [19]:
file_path_candidates = "/content/all_candidates_standard.csv"
try:
    df_candidates = pd.read_csv(file_path_candidates)
    display(df_candidates.head())
    print("\n")
    display(df_candidates.info())
except FileNotFoundError:
    print(f"Error: File not found at {file_path_candidates}. Please check the file path and try again.")

,mission,object_name,disposition,period,planet_radius,star_temp,star_radius,star_mass,discovery_facility
0,Kepler,K00005.01,CANDIDATE,4.780328,7.14,5937.0,1.787,1.197,Kepler
1,Kepler,K00064.01,CANDIDATE,1.951082,10.35,5302.0,2.426,1.258,Kepler
2,Kepler,K00075.01,CANDIDATE,105.881767,10.16,5903.0,2.499,1.301,Kepler
3,Kepler,K00076.01,CANDIDATE,77.479470,10.33,6711.0,2.909,1.657,Kepler
4,Kepler,K00080.01,CANDIDATE,9.250573,96.08,8133.0,3.294,2.053,Kepler




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2663 entries, 0 to 2662
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   mission             2663 non-null   object 
 1   object_name         2663 non-null   object 
 2   disposition         2663 non-null   object 
 3   period              2655 non-null   float64
 4   planet_radius       2558 non-null   float64
 5   star_temp           2559 non-null   float64
 6   star_radius         2558 non-null   float64
 7   star_mass           1875 non-null   float64
 8   discovery_facility  2663 non-null   object 
dtypes: float64(5), object(4)
memory usage: 187.4+ KB


None

## Preprocess the new dataset

### Subtask:
Apply the same preprocessing steps (imputation, scaling, one-hot encoding) used for the training data to the new dataset.


**Reasoning**:
Apply the fitted preprocessor to the new candidate data to prepare it for prediction.



In [20]:
# Select the features from the new dataset
X_candidates = df_candidates[features]

# Apply the same preprocessing steps to the new data
X_candidates_preprocessed = preprocessor.transform(X_candidates)

print(f"Preprocessed candidates data shape: {X_candidates_preprocessed.shape}")

Preprocessed candidates data shape: (2663, 7)


## Predict with keras model

### Subtask:
Use the trained Keras model to make predictions on the preprocessed new data.


**Reasoning**:
Use the trained Keras model to make predictions on the preprocessed new data and store the predictions.



In [21]:
# Use the trained basic_keras_model to predict probabilities on the preprocessed candidate data
keras_candidate_predictions = basic_keras_model.predict(X_candidates_preprocessed)

# Print the shape of the predictions and a few examples
print(f"Keras candidate predictions shape: {keras_candidate_predictions.shape}")
print("Sample Keras predictions:")
print(keras_candidate_predictions[:5])

84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Keras candidate predictions shape: (2663, 1)
Sample Keras predictions:
[[2.0307621e-06]
 [4.1570777e-07]
 [9.7594410e-01]
 [9.7319347e-01]
 [2.7356435e-07]]


## Predict with lightgbm model

### Subtask:
Use the trained LightGBM model to make predictions on the preprocessed new data.


**Reasoning**:
Use the trained LightGBM model to predict probabilities on the preprocessed candidate data and store the predictions.



In [22]:
# Use the trained model_lgb to predict probabilities on the preprocessed candidate data
lgb_candidate_predictions = model_lgb.predict_proba(X_candidates_preprocessed)[:, 1]

# Print the shape of the predictions and a few examples
print(f"LightGBM candidate predictions shape: {lgb_candidate_predictions.shape}")
print("Sample LightGBM predictions:")
print(lgb_candidate_predictions[:5])

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


LightGBM candidate predictions shape: (2663,)
Sample LightGBM predictions:
[0.97752267 0.59924202 0.93188686 0.86356532 0.04386937]


## Display predictions

### Subtask:
Display the predictions from both models.


**Reasoning**:
Create a new DataFrame to store the object names and the predictions from both models and display the first few rows.



In [24]:
# Create a new DataFrame to store object names and predictions
predictions_df = pd.DataFrame({
    'object_name': df_candidates['object_name'],
    'lgb_prediction_score': lgb_candidate_predictions,
    'lgb_prediction': (lgb_candidate_predictions > 0.5).astype(int) # Add the predicted class (0 or 1)
})

# Display the first few rows of the predictions DataFrame
print("Predictions from LightGBM model:")
display(predictions_df.head())

Predictions from LightGBM model:


,object_name,lgb_prediction_score,lgb_prediction
0,K00005.01,0.977523,1
1,K00064.01,0.599242,1
2,K00075.01,0.931887,1
3,K00076.01,0.863565,1
4,K00080.01,0.043869,0


## Summary:

### Data Analysis Key Findings

*   The new dataset contains 2663 candidate entries and 7 features after preprocessing.
*   Both the Keras and LightGBM models successfully generated predictions for all 2663 candidates.
*   The Keras model outputs a single probability score per candidate, while the LightGBM model outputs the probability of the positive class per candidate.

### Insights or Next Steps

*   The predictions from both models can be used to identify potential candidates based on a chosen probability threshold.
*   Consider ensembling the predictions from both models for potentially improved performance.
